# Pràctica 4 PLN: pre-processat de text i extracció de característiques
En aquesta pràctica realitzarem la neteja i pre-processat de diferents conjunts de dades de text.\
Després farem una extracció de característiques com a matriu *sparse* i com a vectors de paraules

### Noms:
Introdueix en aquesta cel·la els noms dels dos integrants del grup:\
*Alumne 1* \
*Alumne 2*

## Part 1: conjunt de textos de "mundocine"
En aquest primer conjunt de dades tenim una sèrie de crítiques de pel·lícules de cinema, emmagatzemades en format XML (una crítica per arxiu). Hem preparat una funció de tipus `generator` que processa el directori on estan els arxius de les crítiques i retorna per cada arxiu XML una tupla amb 4 valors:
 - Nom de la pel·lícula (string)
 - Resum breu de la crítica (string)
 - Text de la crítica (*string*)
 - Valoració de la pel·lícula (*int* d'1 a 5)

In [11]:
import os, re
from xml.dom.minidom import parseString

def parse_folder(path):
    """generator that reads the contents of XML files in a folder
    Returns the <body> of the <review> in each XML file.
    XML files encoded as 'latin-1'"""
    for file in sorted([f for f in os.listdir(path) if f.endswith('.xml')],
                        key=lambda x: int(re.match(r'\d+',x).group())):
        with open(os.path.join(path, file), encoding='latin-1') as f:
            doc=parseString(f.read())

            titulo = doc.documentElement.attributes["title"].value

            btxt = ""
            review_bod = doc.getElementsByTagName("body")
            if len(review_bod) > 0:
                for node in review_bod[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        btxt += node.data + " "

            rtxt = ""
            review_summ = doc.getElementsByTagName("summary")
            if len(review_summ) > 0:
                for node in review_summ[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        rtxt += node.data + " "
                        
            rank = int(doc.documentElement.attributes["rank"].value)
            
            yield titulo, rtxt, btxt, rank


### Exercici
Les crítiques es troben en el directori "critiques" (si no tens el directori, descomprimeix l'arxiu "critiques.zip" que s'entrega en el material de la pràctica. \
Càrrega la primera crítica del directori usant el mètode `next` sobre la funció `parse_folder` en l'objecte `critica`. Mostra els seus 4 valors.

In [12]:
critica = next(parse_folder("criticas"))
critica[0]

'La guerra de los mundos'

## Anàlisi exploratòria
Abans de processar el text calcularem una sèrie de paràmetres de cada crítica. \
Per a això processem cada crítica per a i guardarem els resultats en un objecte `DataFrame` de Colles.\
Com a característica de cada crítica extraurem:
- Títol de la pel·lícula
- Longitud (en caràcters) del resum
- Longitud (en caràcters) del text de la crítica
- Puntuació de la crítica

### Exercici
Completa el codi següent per a generar el `DataFrame`

In [13]:
import pandas as pd

#creamos una lista en blanco
datos = list()

#recorremos las críticas y calculamos sus métricas
for c in parse_folder("criticas"):
    datos.append({
        'título': c[0],
        'LongResumen': str(c[1]),
        'LongCritica': str(c[2]),
        'puntuación': c[3]
    })

resumen = pd.DataFrame(datos)

In [14]:
resumen

,título,LongResumen,LongCritica,puntuación
0,La guerra de los mundos,Hasta los cojones de los yankis,Cada vez me gusta menos el cine de masas. Las ...,1
1,Stars Wars III La venganza de los Sith,La de los sits y el dar vader,"El otro dia fui a ver ""la de los sioux"" como d...",3
2,Los Increibles,¿Por qué las peliculas animadas son mucho mejo...,"Es que no la cagan en ninguna, todas las pelis...",5
3,Spiderman 2,Poniendo a parir una peli que no cuenta nada,Es un dolor esto del cine. Yo ya voy con miedo...,2
4,La casa de cera,"Casquería, terror y cera a partes iguales","Tras una insufrible primera media hora, la cas...",2
5,El internado,Bodrio de terror a la francesa,Joder que bodrio de internado. La peli no empi...,2
6,La llave del mal,Da más miedo la factura de mi móvil,"La llave del mal no es mala película, pero dec...",3
7,May,"May, ¿quieres ser mi amigo?","""May, ¿Quieres ser mi amigo?"" es una de esas p...",4
8,Cinderella Man,Cinta maniquea y mil veces vista,Cinderella Man es como comerse un Whopper con ...,2
9,La Novia Cadaver,Un calco de Pesadilla antes de Navidad.,Yo creía que iba a ver algo nuevo de Tim Burto...,3


## Neteja de text
Prepararem aquest conjunt de textos per a entrenar un model per a predir la puntuació de cada crítica a partir del text de la crítica.\
Realitzarem el següent processament:
- Separar el text en *tokens*
- Eliminar els *tokens* de tipus *stop-word*, signes de puntuació o espais
- Convertir les entitats de tipus `PER` al token *persona*
- Lematizar el text

### Exercici
Completa el codi següent per a realitzar aquestes funcions:

In [15]:
import spacy

nlp = spacy.load("es_core_news_md")

#definimos función de normalizado
def normaliza(texto):
    doc = nlp(texto)
    tokens = #devuelve los tokens que cumplen las condiciones
    palabras = []
    for t in tokens:
        if t.ent_iob_=='B' and t.ent_type_=='PER':
            palabras.append('persona')
        elif t.ent_iob_=='I' and t.ent_type_=='PER':
            continue
        else:
            palabras.append(_._____) #si no es PER añadimos el lema
    salida = #junta todos los tokens en un string
    
    return salida

SyntaxError: invalid syntax (1800721667.py, line 8)

Comprova el seu funcionament en la crítica prèviament descarregada (variable `critica`)

In [ ]:
normaliza(____)

### Possibles millores sobre el processament
Observem els següents problemes:
- Algunes paraules no se separen correctament perquè en el text original estan unides per signes de puntuació.
- La llista de *stop-words* de spaCy no conté 'a','e','i'.
- Alguns lemes mantenen les majúscules.\

Redefinim la funció de normalització per a corregir això:
- Introduïm un espai després de determinats signes de puntuació (".", "?") perquè la divisió en tokens siga correcta
- Filtrem els *tokens* amb una longitud d'1
- Passem a minúscules el lema de cada token

Completa la funció per a realitzar aquestes correccions:

In [ ]:
def normaliza_bis(texto):
    texto = re.sub(r"___", r"___", texto) #añadimos un espacio después de "." y "?"
    doc = nlp(texto)
    tokens = #filtramos los tokens que nos interesan
    palabras = []
    for t in tokens:
        if t.ent_iob_=='B' and t.ent_type_=='PER':
            palabras.append('persona')
        elif t.ent_iob_=='I' and t.ent_type_=='PER':
            continue
        else:
            palabras.append(___) #añadimos lema en minúsculas
    salida = #junta todos los tokens en un string
    
    return salida

Comprova el seu funcionament en la crítica prèviament descarregada (variable `critica`)

In [ ]:
normaliza_bis(____)

### Anàlisi morfològica
En una crítica té molta importància els adjectius utilitzats.\
Crea una funció per a filtrar només els adjectius utilitzats en cada crítica (utilitza el lema de cada adjectiu).

In [ ]:
def extraer_adj(texto):
    texto = re.sub(___, ___, texto) #separamos . y ?
    doc = nlp(texto)
    tokens = #obtenemos lema de todos los tokens de tipo ADJ
    
    return ' '.join(tokens)

Comprova el seu funcionament en la crítica prèviament descarregada (variable `critica`)

In [ ]:
extraer_adj(____)

### Processament de tot el conjunt de dades
Aplicarem aquestes funcions en bloc a tot el conjunt de dades. \
En aquesta ocasió, no farem res amb el text normalitzat sinó que només ho aplicarem per a calcular el núm. de paraules i el núm. d'adjectius de cada crítica.

### Exercici
Completa el codi següent:\
Nota: tingues en compte que per a comptar paraules has de dividir el *string* en espais i comptar el núm. d'elements

In [ ]:
#creamos una lista en blanco
datos = []

#recorremos las críticas y calculamos sus métricas
for c in parse_folder("criticas"):
    datos.append({
        'título': ___,
        'LongResumen': ___,
        'LongCritica': ___,
        'NumPalabras': ___,
        'NumAdj': ___,
        'puntuación': c[3]
    })

resumen = pd.DataFrame(datos)
resumen

## Part 2: Extracció de característiques *sparse*
Anem a calcules les matrius de característiques *bag-of-words* i *tfidf* del conjunt de textos anterior.\
Usarem la llibreria `scikit-learn` per a vectorizar els documents.

In [ ]:
#Para no tener que cargar todas las críticas en memoria,
#creamos un generador que devuelve iterativamente el
#texto procesado de cada crítica

def generaCritica(criticas):
    """Función de tipo generator que devuelve el
    texto normalizado de cada crítica.
    Entrada:
    criticas: objeto 'parse_folder' que itera
    sobre el directio de las críticas
    Salida:
    texto normalizado de cada crítica"""
    for c in criticas:
        yield normaliza_bis(c[2])

Comprova el seu funcionament generant el text normalitzat de la primera crítica

In [ ]:
next(generaCritica(____(____)))

Vectorizem tot el conjunt de dades usant les funcions de `scikit-learn`.\
Aquestes funcions admeten un objecte `generator` com a argument d'entrada.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

criticas = #creamos el objeto generador
BoW_criticas = #entrena y transforma con el corpus de críticas
BoW_criticas

### Exercici
Genera diferents variants de matrius de característiques per al conjunt de les crítiques. Prova amb:
- Matriu TF-IDF
- Matriu BoW amb unigrames i bigrames
- Matriu TF-IDF eliminant les paraules menys freqüents i les més freqüents (mínim de 2 i màxim de 5 documents)
- Mostra quines són les paraules més freqüents eliminades

In [ ]:
#matriz TF-IDF

In [ ]:
#matriz BoW con unigramas y bigramas

In [ ]:
#matriz TF-IDF con min_df=1 y max_df=5

In [ ]:
#palabras más frecuentes a eliminar
#aparecen en el atributo 'stop_words_' del vectorizador

## Word embeddings
Ara calcularem els *word vectors* de les paraules del nostre conjunt de dades, usant la classe `word2vec` de la llibreria `gensim`.\
Aquesta llibreria accepta com a argument d'entrada un objecte `iterador` que generarà el text pre-processament de la següent crítica en la seqüència.\
Usarem les funcions de pre-processament de la llibreria `gensim`.\
Primer definim un objecte de tipus `iterator` per a recórrer les crítiques. Es diferencia d'un simple `generator` que es pot reiniciar la generació de la seqüència (necessari per al model `word2vec`)

In [ ]:
from gensim.utils import simple_preprocess
        
class PreprocesaCriticas(object):
    """Pre-procesa el corpus de críticas con la función 'simple_preprocess'
    de la librería gensim
    Entrada: directorio de críticas
    Salida: iterador sobre las críticas (como lista de tokens)"""
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for c in parse_folder(self.dirname):
            yield simple_preprocess(c[2], deacc=True, min_len=2)

In [ ]:
#instanciamos un objeto para nuestro directorio
criticas = PreprocesaCriticas(____)

Per a provar el seu funcionament amb la primera crítica el convertim en iterable i usem el mètode `next`

In [ ]:
#completar

Al contrari que l'objecte generat amb la funció `generaCriticas` l'objecte de `PreprocesaCriticas` es reinicia cada vegada que iterem. \
Calculem els vectors de paraules de tot el corpus amb el model `word2vec` que accepta com a argument d'entrada un objecte de tipus `iterator` com el creat

In [ ]:
#Cálculo de los vectores de palabras
from gensim.models import Word2Vec

model = Word2Vec(___, #iterador con los documentos
                               size=10,          #tamaño del vector
                               window=5,         #nº de términos adyacentes que usamos para el cálculo
                               min_count=5,      #nº mínimo de apariciones del término para contarlo
                               iter=100
                              )

#una vez entrenado el modelo nos quedamos con los vectores calculados
#si no se van a actualizar los vectores con nuevos documentos
model = model.wv
len(model.vocab)

Seleccionem aleatòriament 25 paraules del conjunt calculat

In [ ]:
import numpy as np

palabras_sample = np.random.choice(model.index2word, 25, replace=False)

In [ ]:
palabras_sample

### Exercici
Comprova com funciona el model buscant les paraules més similars semànticament a "trama", "peli" i "pelicula"

In [ ]:
#palabras similares a 'trama'

In [ ]:
#palabras similares a 'peli'

In [ ]:
#palabras similares a 'pelicula'

### Exercici
Veurem la influència del preprocessament. Per a això alimentarem el model amb les crítiques *preprocesadas amb la funció de normalització sobre `spaCy` (que produeix text lematizat i amb un altre filtrat).\
Per a això cal re-definir l'objecte `iterador` sobre el corpus.

In [ ]:
#comparamos con un modelo que use el preprocesado de spaCy
        
class PreprocesaCriticasSpacy(object):
    """Pre-procesa el corpus de críticas con la función de normalización
    definida con la librería spaCy
    Entrada: directorio de críticas
    Salida: iterador sobre las críticas (como lista de tokens)"""
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for c in parse_folder(self.dirname):
            yield ________

### Exercici
Instància aquesta nova classe per al directori de les crítiques en l'objecte `critiques_spacy` i comprova el seu funcionament sobre la primera crítica

In [ ]:
criticas_spacy = PreprocesaCriticasSpacy(____)

In [ ]:
#prueba a generar la primera crítica

Calcula el model de vectors de paraules amb aquest nou *pre-processat

In [ ]:
modelSpacy = Word2Vec(_____, #iterador con los documentos
                               size=10,          #tamaño del vector
                               window=5,         #nº de términos adyacentes que usamos para el cálculo
                               min_count=5,      #nº mínimo de apariciones del término para contarlo
                               iter=100
                              )

#una vez entrenado el modelo nos quedamos con los vectores calculados
#si no se van a actualizar los vectores con nuevos documentos
modelSpacy = modelSpacy.wv
len(modelSpacy.vocab)

### Pregunta
per què el nou model té un vocabulari amb molts menys termes que el model anterior?

### Visualització de word vectors
Usem una reducció de dimensionalitat t-SNE per a visualitzar un grup de paraules

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

palabras_vectors = model[model.index2word]

#seleccinamos unos pocos términos para visualizarlos entre el conjunto
random_idx = np.random.randint(len(model.index2word), size=5)

tsne = TSNE(n_components=2, random_state=0, n_iter=250, perplexity=2)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(palabras_vectors)

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue', alpha=0.1)

labels = np.array(model.index2word)[random_idx]


T_labels = T[random_idx,:]

plt.scatter(T_labels[:, 0], T_labels[:, 1], c='lime', edgecolors='darkgreen')
for label, x, y in zip(labels, T_labels[:, 0], T_labels[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

In [ ]:
labels

Ara carregarem els vectors per a les mateixes paraules amb el model pre-entrenat GloVe de `spaCy` per a comparar

In [ ]:
palabras_vectors = [nlp.vocab[t].vector for t in model.index2word]

tsne = TSNE(n_components=2, random_state=0, n_iter=250, perplexity=2)
np.set_printoptions(suppress=True)
T = tsne.fit_transform(palabras_vectors)

plt.figure(figsize=(14, 8))
plt.scatter(T[:, 0], T[:, 1], c='steelblue', alpha=0.1)

labels = np.array(model.index2word)[random_idx]


T_labels = T[random_idx,:]

plt.scatter(T_labels[:, 0], T_labels[:, 1], c='lime', edgecolors='darkgreen')
for label, x, y in zip(labels, T_labels[:, 0], T_labels[:, 1]):
    plt.annotate(label, xy=(x+1, y+1), xytext=(0, 0), textcoords='offset points')

## Part 3: Conjunt de Tuits en espanyol
Anem a pre-processar un conjunt de tuits en espanyol etiquetatges amb la seua polaritat

In [ ]:
pd.set_option('display.max_colwidth', None)

# Leemos los datos
df = pd.read_csv('tweets_all.csv', index_col=None)

df.head()

### Exercici
Defineix una funció de normalització que faça les següents tasques:
- Eliminar esments i URL mitjançant un patró RegEx
- Separar el text en *tokens* convertint-los a minúscules, eliminant els que siguen signes de puntuació, espais o dígits
- Eliminar els stop-words d'una llista pròpia passada com a argument
- Eliminar els símbols de puntuació dels tokens (etiquetes, admiracions, etc.)
- Eliminar els tokens d'una longitud menor de 2

In [ ]:
import string
#lista de stop-words específicos de nuestro corpus (aproximación)
stop_words = ['los', 'pero', 'por', 'que', 'una']

patron = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text, stop_words=stop_words):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    y eliminamos signos de puntuación.
    Dejamos tokens en minúsculas.
    Como salida volvemos a convertir los tokens en cadena de texto"""
    text = ___ #elimina menciones y URL
    tokens = nlp(text)
    tokens = ___ #filtra tokens (puntuaciones, espacios y dígitos)
    filtered_tokens = ___ #limpia tokens (signos de puntuación, stop-words y longitud<2)
    filtered_text = ___ #juntam,os como string
    
    return filtered_text

Aplica la funció a tots els tuits (columna 'content') creant una nova columna 'net' del dataframe

In [ ]:
df['limpio'] = ___

In [ ]:
df.head()